### Ejercicio 1:
- De la web `url = "https://books.toscrape.com"` obtén la siguiente información del primer libro:
    - Título
    - Precio
    - Stock
    - Puntuación
    - Descripción
    - Género

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By

/tmp/ipykernel_26722/2409488756.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def get_data(soup):
    
    try:
        title = soup.find("h1").text.strip()
    except: title = np.nan
    
    try:
        price = soup.find("p", class_="price_color").text.strip()
    except: price = np.nan
    
    try:
        in_stock = soup.find("p", class_="instock availability").text.strip()
    except: in_stock = np.nan
    
    try:
        gender = soup.find("ul", class_="breadcrumb").find_all("li")[-2].text.strip()
    except: gender = np.nan
    
    try:
        description = soup.find("article").find("p", recursive=False).text.strip()
    except: description = np.nan
    
    try:
        rating = soup.find("p", class_="star-rating")["class"][-1]
    except: rating = np.nan
            
    return title, price, in_stock, gender, description, rating

### Ejercicio 2:
- Obtén la información que pide el **ejercicio 1** de los 20 primeros libros

_Tip: usa Selenium para navegar por la página_

### Ejercicio 3:
- Obtén la información que pide el ejercicio 1 de todos los libros que hay en la página web.

In [ ]:
base_url = "https://books.toscrape.com/"

driver = webdriver.Chrome()
driver.get(base_url)
driver.maximize_window()

soups = []

for p in range(1,51):

    for i in range(1,21):
        xpath = f'//*[@id="default"]/div/div/div/div/section/div[2]/ol/li[{i}]/article/h3/a'
        element = driver.find_element(By.XPATH, xpath)
        element.click()
        soup = BeautifulSoup(driver.page_source, "html.parser")
        soups.append(soup)
        driver.back()
        
    next_xpath = f'//*[@id="default"]/div/div/div/div/section/div[2]/div/ul/li[3]/a'
    if p == 1:
        next_xpath = f'//*[@id="default"]/div/div/div/div/section/div[2]/div/ul/li[2]/a'
        
    if p != 50:
        element = driver.find_element(By.XPATH, next_xpath)
        element.click()
    
driver.quit()

In [ ]:
data = {
    "title" : [],
    "price" : [],
    "in_stock" :[],
    "gender" : [],
    "description" : [],
    "rating" : [],
}

for soup in soups:
    title, price, stock, gender, description, rating = get_data(soup)
    data["title"].append(title)
    data["price"].append(price)
    data["in_stock"].append(stock)
    data["gender"].append(gender)
    data["description"].append(description)
    data["rating"].append(rating)

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.price = df.price.apply(lambda x: float(x[2:]))

df.in_stock = df.in_stock.apply(lambda x: int(''.join([char for char in x if char.isdigit()])))

df.rating = df.rating.replace({"One" : 1, "Two" : 2, "Three" : 3, "Four" : 4, "Five" : 5})

In [ ]:
df